In [1]:
from matplotlib import pyplot as plt
from rnaseq3 import RNAseq as RNAseq3
from os.path import expanduser as exusr
from CellModels.Cells.IO import CellReader
from CellModels.Cells.Filters import Masks
from CellModels.Clustering.IO import ClusteringReader
import numpy as np
import pandas as pd
import logging

%load_ext autoreload
%autoreload 2

logging.basicConfig(level=logging.ERROR)

ModuleNotFoundError: No module named 'loompy'

In [ ]:
aerts_seq = RNAseq3(exusr('~/Google Drive File Stream/My Drive/Projects/RDN-WDP/contrib/scRNAseq/EyeAntennal_Combined_DG2_v4.csv'))
ariss_seq = RNAseq3(exusr('~/Google Drive File Stream/My Drive/Projects/RDN-WDP/contrib/GSE115476_RAW/GSM3178869_DMS.expr.txt'))
aerts = aerts_seq.data
ariss = ariss_seq.data


In [ ]:
clustered = ClusteringReader.read(exusr('~/Google Drive File Stream/My Drive/Projects/RDN-WDP/processing/clustering/bigc100k6n20r1000_metadata.yml'))
clustered.cells

m = Masks(clustered.cells)

In [ ]:
c = clustered.cells
genes = c.genes
cluster_names = {
    1: 'R8 ato(+)',
    2: 'MF ato(+++)',
    3: 'peripodial',
    4: 'posterior',
    5: 'anterior',
    6: 'MF ato(+)',
    7: 'non-R8'
}

c.loc[m.cells_mf_area & (c[('Cluster', 'ward', 6)] == 4), 'Cluster'] = 7

In [ ]:
from scipy.stats import zscore

gcv = c.set_index(('Cluster', 'ward', 6), append=True) \
    .droplevel('Sample') \
    .rename_axis(index={('Cluster', 'ward', 6): 'Cluster'}) \
    .reorder_levels(['Gene', 'Cluster', 'Nucleus']) \
    .rename(columns={('Measurements', 'Normalized', 'Venus'): 'Normalized'}) \
    .sort_index() \
    .sort_index(axis='columns') \
    .xs(('Measurements', 'Normalized'), axis='columns', drop_level=True) \
    .loc[:, ['Venus']]
gcv['Zscore'] = np.nan

z_aerts = pd.DataFrame()
z_ariss = pd.DataFrame()

for gene in c.genes:
    gcv.loc[gene, 'Zscore'] = zscore(np.log(gcv.loc[gene, 'Venus'] + 1e-100))
    try:
        z_aerts[gene] = zscore(np.log(aerts[gene] + 1e-100))
    except KeyError:
        z_aerts[gene] = np.nan
    try:
        z_ariss[gene] = zscore(np.log(ariss[gene] + 1e-100))
    except KeyError:
        z_ariss[gene] = np.nan

In [ ]:
import matplotlib.pyplot as plt

gcv.index.unique('Cluster')

In [ ]:
genes = []
for gene in c.genes:
    if gene in aerts.columns or gene in ariss.columns:
        genes.append(gene)
if len(genes) % 2 != 0:
    genes.append(None)

In [ ]:
import math
from scipy.stats import ks_2samp

def expression_histogram(gene, show=False, ax=None, bins=20):
    if ax is None:
        fig, ax = plt.subplots()
    else:
        fig = ax.get_figure()
    
    ax.hist(gcv.loc[gene, 'Zscore'], bins=bins, color='C0', histtype='step', density=True, cumulative=True, range=(-2, 10))
    ax.hist(z_aerts[gene], bins=bins, color='C1', histtype='step', density=True, cumulative=True, range=(-2, 10))
    ax.hist(z_ariss[gene], bins=bins, color='C2', histtype='step', density=True, cumulative=True, range=(-2, 10))

    txt = gene    
    ax.text(0.95, 0.95, txt,
        horizontalalignment='right',
        verticalalignment='top',
        fontsize='x-large',
        transform=ax.transAxes)
    
    if show:
        fig.show()
    return fig

fig, axs = plt.subplots(math.ceil(len(genes) / 2), 2, figsize=(15, 105))
for gene, ax in zip(genes, axs.flatten()):
    if gene is not None:
        f = expression_histogram(gene, ax=ax, bins=50)


In [ ]:
def class_histogram(gene, show=False, ax=None, bins=20):
    if ax is None:
        fig, ax = plt.subplots()
    else:
        fig = ax.get_figure()

    for cluster in gcv.index.unique('Cluster'):
        color = 'C' + str(int(cluster - 1))
        ax.hist(gcv.loc[(gene, cluster), 'Zscore'], bins=bins, color=color, histtype='step', density=True, cumulative=True, range=(-2, 10))

    txt = gene
    
    ax.text(0.95, 0.95, txt,
        horizontalalignment='right',
        verticalalignment='top',
        fontsize='x-large',
        transform=ax.transAxes)
    
    if show:
        fig.show()
    return fig

fig, axs = plt.subplots(math.ceil(len(c.genes) / 2), 2, figsize=(15, 105))
for gene, ax in zip(c.genes, axs.flatten()):
    if gene is not None:
        f = class_histogram(gene, ax=ax, bins=50)


In [ ]:
from sklearn.neighbors import KernelDensity
from scipy.integrate import quad_vec

from CellModels.Cells.Filters import QC
from CellModels.Cells.Tools import CleanUp

excluded = [CleanUp.SYNONYMS[x] if x in CleanUp.SYNONYMS else x for x in QC.GENES_BAD]
excluded.append('seq')

densities = {}
genes2 = []
for gene in genes:
    if gene in excluded:
        continue
    else:
        genes2.append(gene)
        densities[gene] = {}
        for cl in gcv.index.unique('Cluster'):
            densities[gene][cl] = KernelDensity(kernel='gaussian', bandwidth=0.25) \
                .fit(gcv.loc[(gene, cl), 'Zscore'].values.reshape(-1, 1))
            

In [ ]:
from scipy.integrate import quad

def map_sc_seq(ds):
    vals = np.ones((ds.shape[0], len(gcv.index.unique('Cluster'))))
    for gene in genes2:
        for j, c in enumerate(gcv.index.unique('Cluster')):
            cache = {}
            for i, v in enumerate(ds[gene]):
                if not v in cache:
                    cache[v] = quad(lambda x: np.exp(densities[gene][c].score_samples([[x]])), v-0.125, v+0.125)[0]
                vals[i][j] = vals[i][j] * cache[v]

    return np.argmax(vals, axis=1)


In [ ]:
aerts['Cluster'] = map_sc_seq(z_aerts) + 1.0
z_aerts['Cluster'] = aerts['Cluster'].values

In [ ]:
ariss['Cluster'] = map_sc_seq(z_ariss) + 1.0
z_ariss['Cluster'] = ariss['Cluster'].values

In [ ]:
def mapped_histogram(df, gene, show=False, ax=None, bins=20):
    if ax is None:
        fig, ax = plt.subplots()
    else:
        fig = ax.get_figure()

    for cluster in df['Cluster'].unique():
        color = 'C' + str(int(cluster - 1.0))
        cs = df['Cluster'] == cluster
        ax.hist(df.loc[cs, gene], bins=bins, color=color, histtype='step', density=True, cumulative=True, range=(-2, 10), ls='-.')
        ax.hist(gcv.loc[(gene, cluster), 'Zscore'], bins=bins, color=color, histtype='step', density=True, cumulative=True, range=(-2, 10))

    txt = gene
    
    ax.text(0.95, 0.95, txt,
        horizontalalignment='right',
        verticalalignment='top',
        fontsize='x-large',
        transform=ax.transAxes)
    
    if show:
        fig.show()
    return fig

fig, axs = plt.subplots(math.ceil(len(c.genes) / 2), 2, figsize=(15, 105))
for gene, ax in zip(c.genes, axs.flatten()):
    if gene is not None:
        f = mapped_histogram(z_aerts, gene, ax=ax, bins=50)


In [ ]:
fig, axs = plt.subplots(math.ceil(len(c.genes) / 2), 2, figsize=(15, 105))
for gene, ax in zip(c.genes, axs.flatten()):
    if gene is not None:
        f = mapped_histogram(z_ariss, gene, ax=ax, bins=50)

In [ ]:
import seaborn as sns

m_aerts = aerts.drop(['Unnamed: 0', 'Cluster'], axis=1).mean(axis=0)
h_aerts = np.log(aerts.drop(['Unnamed: 0'], axis=1) \
                      .groupby('Cluster')
                      .mean() / m_aerts + np.e**-10) \
            .replace(np.nan, 0)
g = sns.clustermap(h_aerts.rename(cluster_names) \
                          .drop('peripodial') \
                          .transpose()
                          .clip(lower=-5, upper=5), cmap="coolwarm", center=0)

In [ ]:
cluster_names

In [ ]:
m_ariss = ariss.drop(['Cluster'], axis=1).mean(axis=0)
h_ariss = np.log(ariss.groupby('Cluster') \
                      .mean() / m_ariss + np.e**-10) \
            .replace(np.nan, 0)
g = sns.clustermap(h_ariss.rename(cluster_names) \
                          .transpose() \
                          .clip(lower=-5, upper=5), cmap="coolwarm", center=0)

In [ ]:
s_aerts = h_aerts.rename(cluster_names).transpose()
s_aerts

In [ ]:
s_aerts.loc[~s_aerts.index.str.startswith('CG')].sort_values('R8 ato(+)').loc['sens']